## GIL (Global Interpreter Lock)

- CPython이 단일 바이트코드를 수행할 때
- Thread가 interrupt하지 못하도록 전역 인터프리터 잠금을 수행
- Thread상황에서 인터프리터가 올바르게 동작하도록

<span style='color:blue'>**그러나..**</span>
- <u>Multithreading 상황에서도 GIL은 한 번에 한 스레드만 동작하게 합니다.</u>
- Multithreading의 장점이 잘 발휘되지 않습니다.

In [39]:
def factorize(number):
    return [i for i in range(1, number+1) if number % i == 0]

import time

numbers = [2139079, 1214759, 1516637, 1852285] * 4
start = time.time()
for number in numbers:
    factorize(number)
end = time.time()
print('Took %.3f seconds' % (end-start))

Took 2.039 seconds


In [40]:
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=4)
start = time.time()
results = list(pool.map(factorize, numbers))
end = time.time()

print('Took %.3f seconds' % (end-start))


Took 2.179 seconds


In [42]:
from concurrent.futures import ProcessPoolExecutor

pool = ProcessPoolExecutor(max_workers=4)
start = time.time()
results = list(pool.map(factorize, numbers))
end = time.time()

print('Took %.3f seconds' % (end-start))


Took 0.647 seconds


만약 IO를 다룬다면?

In [45]:
import select
import time
def slow_systemcall():
    select.select([], [], [], 0.1)

start = time.time()
for _ in range(5):
    slow_systemcall()
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.515 seconds


In [51]:
from concurrent.futures import ThreadPoolExecutor

start = time.time()
with ThreadPoolExecutor(max_workers=4) as e:
    for _ in range(5):
        e.submit(slow_systemcall)
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.210 seconds


In [52]:
from concurrent.futures import ProcessPoolExecutor

start = time.time()
with ProcessPoolExecutor(max_workers=4) as e:
    for _ in range(5):
        e.submit(slow_systemcall)
end = time.time()

print('Took %.3f seconds' % (end-start))

Took 0.224 seconds
